In [1]:
import torch
import io
import numpy as np
import spacy
import math
import json
import random
from spacy.lemmatizer import Lemmatizer
from gensim.models.fasttext import FastText
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
word_vec_dim = 100
seq_len = 15
ope = {
        '+' : 0,
        '-' : 0, 
        '*' : 0,
        '/' : 0
    }

In [3]:
def random_data(data):
    count = 0
    while(count < 500):
        x = random.randint(0,len(data)-1)
        y = random.randint(0,len(data)-1)
        temp = data[x]
        data[x] = data[y]
        data[y] = temp
        count += 1
    return data

In [4]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [5]:
def find_operation(eq):
    for o in eq:
        if(o == '+' or o == '-' or o == '*' or o == '/'):
            return o

In [33]:
def chunck(question):
    question = question.lower()
    question = question.replace("’s", "")
    question = question.replace("'s", "")
    question = lemmatizer(question)
    question_word = []
    numerical_quantity = []
    doc = nlp(question)
    num = []
    for token in doc:
        if token.pos_ == 'NUM':
            num.append(token.text)
        if token.pos_ != 'NUM' and token.pos_ != 'DET' and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and token.text != '-PRON-':
            question_word.append(token.text)
    return question_word, num

In [7]:
def load_vectors(fname):
    data = {}
    file = open(fname, 'r')
    for line in file:
        tokens = line.split(' ')
        X = tokens[1:]
        X = np.array(X)
        X[X == ''] = 0.0
        X = X.astype(np.float)
        data[tokens[0]] = X 
    return data

In [8]:
def similarity(w1, w2):
    x = word2vec[w1]
    y = word2vec[w2]
    return np.dot(x, y)/(np.linalg.norm(x) * np.linalg.norm(y))

In [9]:
one_hot = {
        '+' : 0,
        '-' : 1, 
        '*' : 2,
        '/' : 3
    }
def read(batch):
    b_size = 10
    ques = []
    Y = np.zeros(b_size)
    for q in range(b_size):
        Q = chunck(data[batch*b_size+q]['sQuestion'])[0]
        ques.append(Q)
        ans = find_operation(data[batch*b_size+q]['lEquations'][0])
        Y[q] = one_hot[ans]
        ope[ans] = 1 + ope[ans]
    X = np.zeros((seq_len, b_size, word_vec_dim))
    
    for w in range(seq_len):
        for q in range(b_size):
            if len(ques[q]) > w and ques[q][w] in word2vec:
                v = word2vec.get(ques[q][w])
                for i in range(word_vec_dim):
                    X[w][q][i] = v[i]*1000
    
    y = torch.from_numpy(Y).long()
    x = torch.from_numpy(X).float()
    return x, y

def read_question(ques):
    l = len(ques)
    X = np.zeros((seq_len, l, word_vec_dim))
    for w in range(seq_len):
        for q in range(l):
            if len(ques[q]) > w and ques[q][w] in word2vec:
                v = word2vec.get(ques[q][w])
                for i in range(word_vec_dim):
                    X[w][q][i] = v[i]*1000
    
    x = torch.from_numpy(X).float()
    return x

In [10]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.gru = nn.GRU(input_size = word_vec_dim, hidden_size = 100, num_layers = 1, dropout = 0.3)
        self.fc1 = nn.Linear(100, 4)
        
    def forward(self, x):
        o, z = self.gru(x)
#        return o, z
        x = self.fc1(z[0])
        x = F.softmax(x, dim = 1)
        return x

In [11]:
with open('./DATA/singleop.json') as file:
    data = json.load(file)
data = random_data(data)

nlp = spacy.load("en")

In [12]:
word2vec = load_vectors('./glove.6B/glove.6B.300d.txt')

In [13]:
model = Model()

criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
x, y = read(0)

/usr/local/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [14]:
for epoch in range(40):
    count = 0
    for i in range(19):
        x, l = read(i)
        y = model(x)
        y = y.argmax(dim = 1)
        for j in range(10):
            if l[j] == y[j]:
                count += 1
    print(epoch, count/len(data))
    for batch in range(19):
        inputs, labels = read(batch)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
# model(read(0)[0])

0 0.2117437722419929
1 0.3202846975088968
2 0.39679715302491103
3 0.40213523131672596
4 0.47153024911032027
5 0.47153024911032027
6 0.47686832740213525
7 0.48398576512455516
8 0.47330960854092524
9 0.49466192170818507
10 0.5124555160142349
11 0.5088967971530249
12 0.5
13 0.501779359430605
14 0.5266903914590747
15 0.5249110320284698
16 0.50355871886121
17 0.5249110320284698
18 0.5302491103202847
19 0.5409252669039146
20 0.5338078291814946
21 0.5177935943060499
22 0.5266903914590747
23 0.5249110320284698
24 0.5320284697508897
25 0.50355871886121
26 0.5088967971530249
27 0.5195729537366548
28 0.505338078291815


KeyboardInterrupt: 

In [15]:
count = 0
cadd = 0
csub = 0
cmul = 0
cdiv = 0

for i in range(50):
    x, l = read(i)
    y = model(x)
    y = y.argmax(dim = 1)
    for j in range(10):
        if l[j] == y[j]:
            count += 1
            if y[j] == torch.tensor(0):
                cadd += 1
            if y[j] == torch.tensor(1):
                csub += 1
            if y[j] == torch.tensor(2):
                cmul += 1
            if y[j] == torch.tensor(3):
                cdiv += 1

In [16]:
def read_test():
    file = open('./DATA/q_test.txt', 'r')
    ques = []
    for q in file:
        ques.append(chunck(str(q))[0])
    #print(ques)
    x = read_question(ques)
    file = open('./DATA/eq_test.txt', 'r')
    Y = np.zeros(len(ques))
    count = 0
    for eq in file:
        o = find_operation(eq)
        Y[count] = ope[o]
        count += 1
    y = torch.from_numpy(Y).long()
    return x, y

In [66]:
def find(num, o):
    if(o == torch.tensor([0])):
        print(float(num[1]), '+', float(num[0]))
        return float(num[1]) + float(num[0])
    if(o == torch.tensor([1])):
        print(float(num[0]), '-', float(num[1]))
        return float(num[0]) - float(num[1])
    if(o == torch.tensor([2])):
        print(float(num[1]), '*', float(num[0]))
        return float(num[1]) * float(num[0])
    if(o == torch.tensor([3])):
        print(float(num[0]), '/', float(num[1]))
        return float(num[0]) / float(num[1])

In [67]:
def result(q):
    ques = []
    q, num = chunck(q)
    
    ques.append(q)
    x = read_question(ques)
    print(find(num, model(x).argmax(dim = 1)))

In [68]:
q = "ram has 5 apples and he sold 4. how many apples are left ?"
result(q)

5.0 - 4.0
1.0


In [69]:
q = "ram has 5 apples and shyam gift him 4 apples. how many apples he have now?"
result(q)

4.0 + 5.0
9.0


In [70]:
q = "4 children are playing tennis together. They all brought 6 balls. How many balls do they have total ?"
result(q)

6.0 * 4.0
24.0


In [71]:
q = "he school is planning a field trip. There are 14 students and 2 seats on each school bus. How many buses are needed to take the trip?"
result(q)

14.0 / 2.0
7.0


In [72]:
q = "Betty has 24 oranges stored in boxes. If there are 3 boxes, how many oranges must go in each box?"
result(q)

24.0 / 3.0
8.0


In [74]:
q = "papa gave 5 he ate 3. how many left ?"
result(q)

5.0 - 3.0
2.0


In [77]:
q = "there are total 4 siblings out of the 2 are male rest are female. how many females are there ?"
result(q)

4.0 - 2.0
2.0


In [83]:
q = "chirag has 2 apples and harshit gave him 2 apples. how many apples chirag have"
result(q)

2.0 + 2.0
4.0


In [95]:
q = "what is left when you subtract 2 from 4"
result(q)

4.0 + 2.0
6.0
